In [1]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [2]:
import pandas as pd
import re

df = pd.read_csv("호주_세관_v2(new20).csv")

df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

for i in range(len(df)):
    df_add.append(df_title[i]+""+df_text[i])

df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)

In [5]:
print(len(df_add))

20


In [31]:
import string
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

doc_list = []
for doc in df_add:
    
    #구두점 제거
    doc1 = "".join([i for i in doc if i not in string.punctuation]).strip()
    
    #숫자 제거
    doc2 = "".join([i for i in doc1 if not i.isdigit()])
    
    #월 제거
    month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
            'september', 'october', 'november', 'december', 'jan', 'feb', 'mar', 'apr',
             'may', 'jun','jul', 'aug', 'sep', 'oct', 'nov', 'dec']   

    doc3 = " ".join([i for i in doc2.split() if i not in month])
    doc_list.append(doc3)

In [32]:
n_gram_range = (2, 2)
stop_words = "english"

candidates_list = []
for doc in doc_list:
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
    candidates = count.get_feature_names_out()
    
    candidates_list.append(candidates)

In [44]:
#문서 전체와 문서에서 추출한 키워드 수치화
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embeddings = []
candidates_embeddings = []
for i in range(len(doc_list)):
    doc_embeddings.append(model.encode([doc_list[i]])) #전체 문서
    candidates_embeddings.append(model.encode(candidates_list[i]))#추출한 bigram

In [ ]:
#문서와 가장 유사한 키워드 추출
doc_bigram_keywords = []
top_n = 20

for 

distances = cosine_similarity(doc_embedding, candidate_embeddings)
doc_bigram_keyword = [candidates[index] for index in distances.argsort()[0][-top_n:]]
doc_bigram_keywords.append(doc_bigram_keyword)

print(len(doc_bigram_keywords))
print(doc_bigram_keywords)